In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from prob2utils_skeleton import train_model, get_err
from surprise import AlgoBase
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import matrix_factorization
from surprise import accuracy
from surprise.model_selection import train_test_split

In [22]:
movies = pd.read_table('data/movies.txt',names=["Movie Id", "Movie Title", "Unknown", "Action", "Adventure", "Animation", "Childrens", "Comedy", "Crime", "Documentary","Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"])
np.save("movie", movies)

data = pd.read_table('data/data.txt', names=["user", "movie", "rating"])
np.save("data", data)

train = pd.read_table('data/train.txt',  names=["user", "movie", "rating"])
np.save("train", train)

test = pd.read_table('data/test.txt', names=["user", "movie", "rating"])
np.save("test", test)

In [67]:
reader = Reader(rating_scale=(1, 5))
#trainset = Dataset.load_from_df(train, reader=reader)
#testset = Dataset.load_from_df(test, reader=reader)
ydata = Dataset.load_from_df(data, reader=reader)
trainset, testset = train_test_split(ydata, test_size=0.1)

In [56]:
filterer = matrix_factorization.SVD()
filterer.fit(trainset)

## Calculate training and test errors. Lambda = 0.02 .

In [69]:
u = filterer.pu
v = filterer.qi
ubias = filterer.bu
vbias = filterer.bi
print(u.shape, ubias.shape, v.shape, vbias.shape)
print(len(testset))

(943, 100) (943,) (1666, 100) (1666,)
10000


In [63]:
def get_err(U, V, BU, BV, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    ratings = np.zeros(len(Y))
    for n in np.arange(len(Y)):
        ratings[n] = Y[n][2]
    mu = np.mean(ratings)
    err = 0.0    
    
    err += 0.5*reg*(np.sum(U**2) + np.sum(V**2) + np.sum(BU**2) + np.sum(BV**2))
    for n in np.arange(len(Y)):
        i = Y[n][0] - 1 #Columns start at 1, not zero.
        j = Y[n][1] - 1 #Columns start at 1, not zero.
        #print(U.shape, V.shape)
        err += 0.5*((ratings[n] - mu) - (np.dot(U[i,:],V.T[:,j]) + BU[i] + BV[j]))**2

    return err

In [70]:
trainerr = get_err(u, v, ubias, vbias, testset, reg=0.02)

IndexError: index 1674 is out of bounds for axis 1 with size 1666

In [32]:
np.savetxt('u_shelf.txt', u)
np.savetxt('v_shelf.txt', v)
np.savetxt('ubias_shelf.txt', ubias)
np.savetxt('vbias_shelf.txt', vbias)